#  T.A. ACTUALITÉS Africaines [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ERntyn-841vjfBseIWbFPyme_VM08kcm?usp=sharing) 

Ce notebook est conçu pour entraîner un modèle de TA (Traduction Automatique) en affinant (fine-tuning) les modèles pré-entraînés sur quelques milliers de phrases. Il prend en charge les architectures de type M2M100, MBART50, MT5 et ByT5. 

##### Sections:

Il y a trois sections dans ce notebook :

1. Installations : installation des dépendances appropriées
2. Importations : importation de tous les modules nécessaires, en particulier Transformers et Sacrebleu.
3. Entraînement : C'est ici que se déroule le processus de formation proprement dit


## 1. Installations

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install sacrebleu

     |████████████████████████████████| 3.5 MB 13.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 6.8 MB 50.0 MB/s 
     |████████████████████████████████| 596 kB 29.7 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 29.7 MB/s 
     |████████████████████████████████| 1.1 MB 64.7 MB/s 
     |████████████████████████████████| 133 kB 69.1 MB/s 
     |████████████████████████████████| 243 kB 71.2 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 144 kB 74.5 MB/s 
     |████████████████████████████████| 271 kB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 33.0 MB/s 
     |████████████████████████████████| 90 kB 9.9 MB/s 


## 2. Importations

In [ ]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint

logger = logging.getLogger(__name__)

MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast, M2M100Tokenizer]


## 3. Code d'entraînement

In [ ]:
def start_training(model_args, data_args, training_args):
    # Voir tous les arguments possibles dans src/transformers/training_args.py
    # ou en utilisant l'option --help pour ce script.
    # Nous gardons maintenant des ensembles distincts d'arguments (args), 
    # pour une séparation plus nette des tâches.

    # Configuration de la journalisation (le logging)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )

    log_level = training_args.get_process_log_level()
    logger.setLevel(log_level)
    datasets.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()

    # Ouvrir une session et traiter le petit résumé :
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")

    if data_args.source_prefix is None and model_args.model_name_or_path in [
        "t5-small",
        "t5-base",
        "t5-large",
        "t5-3b",
        "t5-11b",
    ]:
        logger.warning(
            "You're running a t5 model but didn't provide a source prefix, which is expected, e.g. with "
            "`--source_prefix 'translate English to German: ' `"
        )

    # Détection du dernier point de contrôle.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Définir le seed avant d'initialiser le modèle.
    set_seed(training_args.seed)

    # Obtenir les jeux de données : vous pouvez fournir vos propres fichiers JSON d'entraînement et 
    # d'évaluation (voir ci-dessous) ou fournir simplement le nom de l'un des ensembles de données 
    # publics disponibles sur le hub à l'adresse https://huggingface.co/datasets/.
    # Le jeu de données sera téléchargé automatiquement depuis le hub des jeux de données).
    #
    # Pour la traduction, seuls les fichiers JSON sont pris en charge, avec un champ nommé "translation" 
    # contenant deux clés pour les langues source et cible (sauf si vous adaptez ce qui suit).
    #
    # Dans le cadre des entraînements distribués, la fonction load_dataset garantit qu'un seul processus 
    # local peut télécharger simultanément l'ensemble de données.
    if data_args.dataset_name is not None:
        # Téléchargement et chargement d'un ensemble de données à partir du hub.
        raw_datasets = load_dataset(
            data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir
        )
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
            extension = data_args.train_file.split(".")[-1]
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
            extension = data_args.validation_file.split(".")[-1]
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
            extension = data_args.test_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)
    # Pour en savoir plus sur le chargement de tout type de jeux de données standard ou personnalisés 
    # (à partir de fichiers, de dictionnaires python, de DataFrame de Pandas, etc.), voir:
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Chargement du modèle pré-entraîné et du tokenizer
    #
    # Entraînement distribué :
    # La méthode .from_pretrained garantit qu'un seul processus local peut concurremment
    # télécharger le modèle et le vocabulaire.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    model.resize_token_embeddings(len(tokenizer))

    # Définir decoder_start_token_id
    if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
        if isinstance(tokenizer, MBartTokenizer):
            model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.target_lang]
        else:
            model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(data_args.target_lang)

    if model.config.decoder_start_token_id is None:
        raise ValueError("Assurez-vous que `config.decoder_start_token_id` soit correctement défini.")

    prefix = data_args.source_prefix if data_args.source_prefix is not None else ""

    # Prétraitement des jeux de données.
    # Nous devons tokeniser les entrées et les cibles.
    if training_args.do_train:
        column_names = raw_datasets["train"].column_names
    elif training_args.do_eval:
        column_names = raw_datasets["validation"].column_names
    elif training_args.do_predict:
        column_names = raw_datasets["test"].column_names
    else:
        logger.info("Aucune action n'est nécessaire. Veuillez passer `do_train`, `do_eval` et/ou `do_predict`.")
        return

    # Pour la traduction, nous définissons les codes de nos langues source et cible (seulement utile pour mBART, 
    # les autres vont ignorer ces attributs).
    if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):
        assert data_args.target_lang is not None and data_args.source_lang is not None, (
            f"{tokenizer.__class__.__name__} is a multilingual tokenizer which requires --source_lang and "
            "--target_lang arguments."
        )

        tokenizer.src_lang = data_args.source_lang
        tokenizer.tgt_lang = data_args.target_lang

        # Pour les modèles de traduction multilingues comme mBART-50 et M2M100, 
        # nous devons forcer le jeton de la langue cible
        # comme premier jeton généré. Nous demandons à l'utilisateur d'indiquer
        # explicitement --forced_bos_token comme argument.
        forced_bos_token_id = (
            tokenizer.lang_code_to_id[data_args.forced_bos_token] if data_args.forced_bos_token is not None else None
        )
        model.config.forced_bos_token_id = forced_bos_token_id

    # Obtention des codes de langue pour la source/cible.
    source_lang = data_args.source_lang.split("_")[0]
    target_lang = data_args.target_lang.split("_")[0]

    # Définir temporairement max_target_length (la longueur maximale de la cible) pour l'entrainement'.
    max_target_length = data_args.max_target_length
    padding = "max_length" if data_args.pad_to_max_length else False

    if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
        logger.warning(
            "label_smoothing est activé mais la méthode `prepare_decoder_input_ids_from_labels` n'est pas définie pour"
            f"`{model.__class__.__name__}`. Ceci conduira à ce que la perte soit calculée deux fois et occupera plus de mémoire."
        )

    def preprocess_function(examples):
        inputs = [ex[source_lang] for ex in examples["translation"]]
        targets = [ex[target_lang] for ex in examples["translation"]]
        inputs = [prefix + inp for inp in inputs]
        model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

        # Configuration du tokenizer pour les cibles
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        # Si nous faisons du remplissage ici, il faut remplacer tous les tokenizer.pad_token_id dans les étiquettes
        # par la valeur -100 lorsque nous voulons ignorer le remplissage dans la perte.
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    if training_args.do_train:
        if "train" not in raw_datasets:
            raise ValueError("--do_train nécessite un jeu de données d'entraînement")
        train_dataset = raw_datasets["train"]
        if data_args.max_train_samples is not None:
            train_dataset = train_dataset.select(range(data_args.max_train_samples))
        with training_args.main_process_first(desc="train dataset map pre-processing"):
            train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

    if training_args.do_eval:
        max_target_length = data_args.val_max_target_length
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval nécessite un jeu de données de validation")
        eval_dataset = raw_datasets["validation"]
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
        with training_args.main_process_first(desc="validation dataset map pre-processing"):
            eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on validation dataset",
            )

    if training_args.do_predict:
        max_target_length = data_args.val_max_target_length
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = raw_datasets["test"]
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))
        with training_args.main_process_first(desc="prediction dataset map pre-processing"):
            predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )

    # Collecteur de données
    label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    if data_args.pad_to_max_length:
        data_collator = default_data_collator
    else:
        data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=8 if training_args.fp16 else None,
        )

    # Métrique
    metric = load_metric("sacrebleu")

    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        if data_args.ignore_pad_token_for_loss:
            # Remplacer -100 dans les étiquettes car nous ne pouvons pas les décoder.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Quelques post-traitements simples
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

    # Initialisation de notre Trainer (formateur)
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.predict_with_generate else None,
    )

    # Entraînement
    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # Sauvegarde également du tokenizer pour faciliter le téléchargement.

        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Évaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Évaluation ***")

        metrics = trainer.evaluate(
            max_length=data_args.val_max_target_length, num_beams=data_args.num_beams, metric_key_prefix="eval"
        )
        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    if training_args.do_predict:
        logger.info("*** Prédiction ***")

        predict_results = trainer.predict(
            predict_dataset,
            metric_key_prefix="predict",
            max_length=data_args.val_max_target_length,
            num_beams=data_args.num_beams,
        )
        metrics = predict_results.metrics
        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        if trainer.is_world_process_zero():
            if training_args.predict_with_generate:
                predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                predictions = [pred.strip().replace("\n"," ") for pred in predictions]
                output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
                with open(output_prediction_file, "w", encoding="utf-8") as writer:
                    writer.write("\n".join(predictions))

    if training_args.push_to_hub:
        kwargs = {"finetuned_from": model_args.model_name_or_path, "tasks": "translation"}
        if data_args.dataset_name is not None:
            kwargs["dataset_tags"] = data_args.dataset_name
            if data_args.dataset_config_name is not None:
                kwargs["dataset_args"] = data_args.dataset_config_name
                kwargs["dataset"] = f"{data_args.dataset_name} {data_args.dataset_config_name}"
            else:
                kwargs["dataset"] = data_args.dataset_name

        languages = [l for l in [data_args.source_lang, data_args.target_lang] if l is not None]
        if len(languages) > 0:
            kwargs["language"] = languages

        trainer.push_to_hub(**kwargs)

    return results

## 4. Arguments de formation

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments relatifs au model/config/tokenizer à partir duquel nous allons procéder à un réglage fin (fine-tuning).
    """

    model_name_or_path: str = field(
        metadata={"help": "Chemin vers le modèle pré-entraîné ou l'identifiant du modèle à partir de huggingface.co/models."}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Nom de la configuration préformée ou chemin d'accès si celui-ci est différent de model_name."}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Nom du tokenizer pré-entraîné ou chemin si différent de model_name."}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Où stocker les modèles pré-formés téléchargés à partir de huggingface.co."},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Utilisation ou non du fast tokenizer (fourni par la bibliothèque tokenizers)."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "La version spécifique du modèle à utiliser (peut être un nom de branche, un nom de tag ou un identifiant de commit)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Le jeton généré lors de l'exécution de la commande `transformers-cli login` (nécessaire pour utiliser ce script) sera utilisé "
            "avec des modèles privés)."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments relatifs aux données que nous allons introduire dans notre modèle pour la formation et l'évaluation.
    """

    source_lang: str = field(default=None, metadata={"help": "Identifiant de la langue source pour la traduction."})
    target_lang: str = field(default=None, metadata={"help": "Identifiant de la langue cible pour la traduction."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "Le nom du jeu de données à utiliser (via la bibliothèque datasets)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "Le nom de configuration du jeu de données à utiliser (via la bibliothèque datasets)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "Le fichier de données d'entraînement d'entrée (un jsonlines)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "Un fichier de données d'évaluation d'entrée facultatif pour évaluer les métriques (sacreblue) sur "
            "un fichier jsonlines."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "Un fichier de données de test d'entrée facultatif pour évaluer les métriques (sacreblue) sur " "un fichier jsonlines."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Écraser les ensembles de formation et d'évaluation mis en cache"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "Le nombre de processus à utiliser pour le prétraitement."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": "La longueur totale maximale de la séquence d'entrée après tokénisation. Les séquences plus longues "
            "seront tronquées, les séquences plus courtes seront complétées."
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": "La longueur totale maximale de la séquence pour le texte cible après tokénisation. Les séquences plus longues "
            "seront tronquées, les séquences plus courtes seront complétées."
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "La longueur totale maximale de la séquence pour le texte cible de validation après tokénisation. Les séquences plus longues "
            "seront tronquées, les séquences plus courtes seront complétées. La valeur par défaut sera `max_target_length`."
            "Cet argument est également utilisé pour remplacer le paramètre ``max_length`` de ``model.generate``, qui est utilisé "
            "pendant ``evaluate`` et ``predict``."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "S'il faut remplir tous les échantillons pour modéliser la longueur maximale de la phrase. "
            "Si Faux, les échantillons seront remplis dynamiquement lors du regroupement (batching) jusqu'à la longueur maximale du lot. "
            "Plus efficace sur le GPU mais très mauvais pour le TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "Pour des besoins de débogage ou pour accélérer l'apprentissage, tronquez le nombre d'exemples d'apprentissage "
            "à cette valeur si elle est définie."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "Pour des besoins de débogage ou pour accélérer la formation, tronquez le nombre d'exemples d'évaluation à cette valeur "
            "à cette valeur si elle est définie."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "Pour des besoins de débogage ou pour accélérer la formation, tronquez le nombre d'exemples de prédiction à cette valeur "
            "à cette valeur si elle est définie."
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": "Nombre de faisceaux (beams) à utiliser pour l'évaluation. Cet argument sera passé à ``model.generate``, "
            "qui est utilisé pendant ``evaluate`` et ``predict``."
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Indique s'il faut ignorer ou non les jetons correspondant aux étiquettes rembourrées dans le calcul de la perte."
        },
    )
    source_prefix: Optional[str] = field(
        default=None, metadata={"help": "Un préfixe à ajouter avant chaque texte source (utile pour les modèles T5)."}
    )
    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": "Le jeton à forcer en tant que premier jeton généré après le :obj:`decoder_start_token_id`."
            "Utile pour les modèles multilingues comme :doc:`mBART <../model_doc/mbart>` où le premier jeton généré "
            "doit être le token de la langue cible (habituellement, c'est le token de la langue cible)."
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Besoin d'un nom de jeu de données ou d'un fichier de formation/validation.")
        elif self.source_lang is None or self.target_lang is None:
            raise ValueError("Nécessité de spécifier la langue source et la langue cible.")

        if self.train_file is not None:
            extension = self.train_file.split(".")[-1]
            assert extension == "json", "`train_file` devrait être un fichier json."
        if self.validation_file is not None:
            extension = self.validation_file.split(".")[-1]
            assert extension == "json", "`validation_file` devrait être un fichier json."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length

In [ ]:
def get_args(args_dict):
  parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments)) 

  list_args = []
  for arg in args_dict:
    list_args+=['--'+arg, args_dict[arg]] 

  print(list_args)
  #list_args=['--model_name_or_path', 'google/mt5-small','--output_dir', 'mt5_small_zu_en', '--train_file', 'data/json_files/en_zu_lafand/train.json',
  #         '--max_source_length','128', '--max_target_length', '128', '--source_lang','zu','--target_lang', 'en']
  
  model_args, data_args, training_args = parser.parse_args_into_dataclasses(list_args)

  return model_args, data_args, training_args

### Ajout de tous les hyperparamètres nécessaires, du nom du modèle ainsi que du chemin d'accès aux jeux de formation, de développement et de test.

**MODIFIER LA CELLULE SUIVANTE**

In [ ]:
!git clone https://github.com/masakhane-io/lafand-mt.git

Cloning into 'lafand-mt'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 245 (delta 10), reused 0 (delta 0), pack-reused 206
Receiving objects: 100% (245/245), 32.69 MiB | 21.21 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [ ]:
args_dict = {
    "output_dir": "m2m100_small_en_zu",
    "model_name_or_path": "facebook/m2m100_418M",
    "train_file": "lafand-mt/data/json_files/en-zu-news/train.json",
    "validation_file": "lafand-mt/data/json_files/en-zu-news/dev.json",
    "test_file": "lafand-mt/data/json_files/en-zu-news/test.json",
    "source_lang": "en",
    "target_lang": "zu",

    "max_source_length": "128",
    "max_target_length": "128",
    "num_train_epochs": "3",
    "per_device_train_batch_size": "4",
    "per_device_eval_batch_size": "4",
    "num_beams": "5",
    "save_steps": "10000",
    "seed": "65",

    "do_train": "True",
    "do_eval": "True",
    "do_predict":"True",
    "predict_with_generate": "True",
    "overwrite_output_dir": "True",
   

    # optionnel
    # pour mt5
    # "source_prefix": "traduire le zoulou en anglais : ",
    # pour m2m100
    "forced_bos_token": "zu",
    # pour mBART50
    #"forced_bos_token": "en_XX", # le code de langue a _[code pays]

}

In [ ]:
model_args, data_args, training_args = get_args(args_dict)

['--output_dir', 'm2m100_small_zu_en', '--model_name_or_path', 'facebook/m2m100_418M', '--train_file', 'lafand-mt/data/json_files/en-zu-news/train.json', '--validation_file', 'lafand-mt/data/json_files/en-zu-news/dev.json', '--test_file', 'lafand-mt/data/json_files/en-zu-news/test.json', '--source_lang', 'en', '--target_lang', 'zu', '--max_source_length', '128', '--max_target_length', '128', '--num_train_epochs', '3', '--per_device_train_batch_size', '4', '--per_device_eval_batch_size', '4', '--num_beams', '5', '--save_steps', '10000', '--seed', '65', '--do_train', 'True', '--do_eval', 'True', '--do_predict', 'True', '--predict_with_generate', 'True', '--overwrite_output_dir', 'True', '--forced_bos_token', 'zu']


In [ ]:
print(model_args)

ModelArguments(model_name_or_path='facebook/m2m100_418M', config_name=None, tokenizer_name=None, cache_dir=None, use_fast_tokenizer=True, model_revision='main', use_auth_token=False)


**DÉBUT DE L'ENTRAÎNEMENT**

In [ ]:
start_training(model_args, data_args, training_args)



01/30/2022 19:08:51 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/30/2022 19:08:51 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<

  0%|          | 0/3 [00:00<?, ?it/s]

01/30/2022 19:08:51 - INFO - datasets.utils.download_manager - Downloading took 0.0 min
01/30/2022 19:08:51 - INFO - datasets.utils.download_manager - Checksum Computation took 0.0 min


  0%|          | 0/3 [00:00<?, ?it/s]

01/30/2022 19:08:51 - INFO - datasets.utils.info_utils - Unable to verify checksums.
01/30/2022 19:08:51 - INFO - datasets.builder - Generating split train
01/30/2022 19:08:51 - INFO - datasets.builder - Generating split validation
01/30/2022 19:08:51 - INFO - datasets.builder - Generating split test
01/30/2022 19:08:51 - INFO - datasets.utils.info_utils - Unable to verify splits sizes.
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-84a84a53663a1ae7/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


...

[INFO|trainer.py:2340] 2022-01-30 19:31:53,720 >> ***** Running Evaluation *****
[INFO|trainer.py:2342] 2022-01-30 19:31:53,722 >>   Num examples = 1239
[INFO|trainer.py:2345] 2022-01-30 19:31:53,724 >>   Batch size = 4


01/30/2022 19:42:53 - INFO - datasets.metric - Removing /root/.cache/huggingface/metrics/sacrebleu/default/default_experiment-1-0.arrow
***** eval metrics *****
  epoch                   =        3.0
  eval_bleu               =     8.4881
  eval_gen_len            =    46.7393
  eval_loss               =     2.7475
  eval_runtime            = 0:11:00.12
  eval_samples            =       1239
  eval_samples_per_second =      1.877
  eval_steps_per_second   =       0.47
01/30/2022 19:42:53 - INFO - __main__ - *** Predict ***


[INFO|trainer.py:2340] 2022-01-30 19:42:53,856 >> ***** Running Prediction *****
[INFO|trainer.py:2342] 2022-01-30 19:42:53,859 >>   Num examples = 998
[INFO|trainer.py:2345] 2022-01-30 19:42:53,862 >>   Batch size = 4


01/30/2022 19:52:21 - INFO - datasets.metric - Removing /root/.cache/huggingface/metrics/sacrebleu/default/default_experiment-1-0.arrow
***** predict metrics *****
  predict_bleu               =    11.9544
  predict_gen_len            =    50.4269
  predict_loss               =     2.3962
  predict_runtime            = 0:09:27.48
  predict_samples            =        998
  predict_samples_per_second =      1.759
  predict_steps_per_second   =      0.441


{}